In [1]:
import yaml
from pprint import pprint

with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)
pprint(config)

{'dataset': {'augment': True,
             'batch_size': 4,
             'data_dir': '/Dataset',
             'n_threads': 16},
 'model': {'name': 'rcan'},
 'train': {'epochs': 100,
           'loss': 'edsr',
           'optimizer': {'lr': 0.002, 'name': 'Adam'},
           'scheduler': {'gamma': 0.5, 'name': 'StepLR', 'step_size': 5}}}


In [2]:
from data.loader import get_data_loader
from utility import check_dataloader

train_loader, val_loader = get_data_loader(config, mode='train', val_split=0.2)
# check_dataloader(val_loader)

Total Images: 5093


In [3]:
test_loader = get_data_loader(config, mode='test')
check_dataloader(test_loader)

Total Images: 843
Batch size: 12
Batch 1


In [4]:
import os
print(os.getcwd())

/root


In [5]:
# from model.redcnn import REDCNN
# from trainer import Trainer

from trainer import Trainer
from src.model.rcan import RCAN
from option import args

rcan = RCAN(args)
trainer = Trainer(model=rcan, 
                   model_name=config['model']['name'], 
                   config=config, 
                   args=args,
                   train_loader=train_loader, 
                   val_loader=val_loader)

# redcnn = REDCNN()
# trainer = Trainer(model=redcnn, 
#                   model_name=config['model']['name'], 
#                   config=config, 
#                   train_loader=train_loader, 
#                   val_loader=val_loader)

Preparing loss function:


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


1.000 * L1
0.005 * VGG54
0.010 * RGAN


In [9]:
import torch
torch.autograd.set_detect_anomaly(True)
trainer.train_and_eval()

OutOfMemoryError: CUDA out of memory. Tried to allocate 256.00 MiB. GPU 

In [ ]:
from utility import *

model = RCAN(args).to('cuda')
model.load_state_dict(torch.load('saved_model/rcan.pth'))

distorted_images_list, predicted_images_list, ground_truth_images_list = inference('cuda', model, test_loader)
inference_loss(predicted_images_list, ground_truth_images_list, loss='l1')
print()
inference_loss(predicted_images_list, ground_truth_images_list, loss='l2')

In [ ]:
calculate_psnr_ssim(predicted_images_list, ground_truth_images_list)

In [ ]:
plot_images(distorted_images_list, predicted_images_list, ground_truth_images_list, '10_180', 5)

In [ ]:
pimages(distorted_images_list, predicted_images_list, ground_truth_images_list, idx=0)

In [ ]:
distortion_list = ['10_180', '10_360', '10_720', '25_180', '25_360', '25_720', 
                       '50_180', '50_360', '50_720', '100_180', '100_360', '100_720']
pimages(distorted_images_list, predicted_images_list, ground_truth_images_list, distortion_list[11], idx=0)
